<div style="text-align: right"><b> Ana Valentina López Chacón & Camila Palomeque </b></div>
<div style="text-align: right"><b> Herramientas y Aplicaciones de la Inteligencia Artificial, 2024 - 2025 </b></div>

## **Proyecto: Detección de Argumentos**

In [1]:
!pip install langchain pymupdf openai --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from typing import List
from pydantic import BaseModel, Field, ValidationError
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
import requests
import json
import re
from openai import OpenAI
import openai
import httpx

In [3]:
import fitz 

def extract_text_from_pdf(pdf_path, page_list):
    doc = fitz.open(pdf_path)
    text = ""
    for page in page_list:
        if 0 <= page < len(doc):
            text += doc[page].get_text()
    return text


In [5]:
client = OpenAI(
    base_url = 'https://api.poligpt.upv.es',  
    api_key = 'sk-Icbf-5FyeV0QcLWBC9SNEA'     
)

timeout = httpx.Timeout(60.0, connect=30.0)

# chat_completion = client.chat.completions.create(
#         messages = [
#             {'role': 'system', 'content': 'You are an argument mining assistant.'},
#             {'role': 'user', 'content': 'Say hello'}
#         ],
#         model = "poligpt",
#         temperature = 0,
#         # timeout = timeout
#     )

# result = chat_completion.choices[0].message.content
# result

In [6]:
class ArgumentResponse(BaseModel):
    in_favor: list[str] = Field(..., description= "List of arguments (strings) in favor of the topic extracted directly from the text.")
    against: list[str] = Field(..., description = "List of arguments (strings) against the topic extracted from the text.")

pydantic_parser = PydanticOutputParser(pydantic_object=ArgumentResponse)

def extract_arguments_json(text: str, topic: str) -> ArgumentResponse:
    format_instructions = pydantic_parser.get_format_instructions()

    prompt = PromptTemplate(
        template=(
            "You are an argument mining assistant.\n\n"
            "I want to extract the arguments in agreement or inference (in favor) and disagreement or conflict (against) out of the Sustainable Development Goals agenda 2030 (SGD).\n\n"
            "Extract arguments in favor and against ONLY in regard to this specific Goal: \"{topic}\" directly from the text below.\n\n"
            "Text:\n\"\"\"\n{text}\n\"\"\""
            "Respond ONLY with a JSON object like this:\n\n"
            "{format_instructions}\n\n"
            "Do not include markdown (```), explanations, or any extra text. Just output the raw JSON object."
        ),
        input_variables=["text", "topic"],
        partial_variables={"format_instructions": format_instructions},
    )

    final_prompt = prompt.format_prompt(text=text, topic=topic).to_string()

    chat_completion = client.chat.completions.create(
        messages = [
            {'role': 'system', 'content': 'You are an argument mining assistant.'},
            {'role': 'user', 'content': final_prompt}
        ],
        model = "poligpt",
        temperature = 0,
        # timeout = timeout
    )

    raw_output = chat_completion.choices[0].message.content

    print("Model Output:", raw_output)

    return pydantic_parser.parse(raw_output)

### **Goal 1: End poverty in all its forms everywhere**

In [7]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g1 = extract_text_from_pdf(pdf_path, [9, 10])
print(text_g1)

The Sustainable Development Goals Report 2024
8
No poverty
•	
The COVID-19 pandemic and subsequent shocks from 2020 to 2022 
have hampered global efforts to eradicate extreme poverty. The global 
extreme poverty rate increased in 2020 for the first time in decades, 
setting back progress by three years. Since then, recovery has been 
uneven, with low-income countries lagging behind. By 2030, 590 million 
people may still live in extreme poverty if current trends persist. Without 
a substantial acceleration in poverty reduction, fewer than 3 in 10 
countries are expected to halve national poverty by 2030.
•	
	Despite increasing efforts and commitments to expand social protection 
programmes, significant coverage gaps left 1.4 billion children uncovered 
in 2023. 
•	
	Climate change is hindering poverty reduction, and disasters result in 
millions of households becoming poor or remaining trapped in poverty. 
•	
Ending poverty requires a wide-ranging approach that combines 
comprehensive 

In [8]:
topic_g1 = "End poverty in all its forms everywhere"

result = extract_arguments_json(text_g1, topic_g1)
print(result)

Model Output: {"in_favor": [
  "Efforts to reduce disaster impacts and risks have been made",
  "Government spending on essential services is increasing",
  "Advanced economies are outpacing emerging market and developing economies in social protection spending",
  "There has been a slight uptick in the shares of government spending on essential services over the past two decades"
], "against": [
  "Disaster-related economic losses remain high, with average annual direct economic losses exceeding $115 billion worldwide",
  "Low-income countries suffer disproportionately from disasters, with disaster-related economic losses over six times the global average",
  "There is a chronic gap between advanced and developing economies in government spending on essential services",
  "The pandemic has temporarily skewed trends in essential services spending, with potential long-term impacts on human capital"
]}
in_favor=['Efforts to reduce disaster impacts and risks have been made', 'Government s

In [9]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal1_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 2: End hunger, achieve food security and improved nutrition and promote sustainable agriculture**

In [10]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g2 = extract_text_from_pdf(pdf_path, [11, 12])
print(text_g2)

The Sustainable Development Goals Report 2024
10
Zero hunger
•	
After rising sharply in the wake of the COVID-19 pandemic, global 
hunger and food insecurity remained persistently high and almost 
unchanged for three years. In 2023, about 733 million people faced 
hunger, and 2.33 billion people experienced moderate to severe food 
insecurity. Despite progress, 148 million children under age 5 suffered 
from stunting in 2022. If current trends persist, one in five children under 
age 5 will be affected by stunting in 2030.
•	
	In 2022, almost 60 per cent of countries worldwide faced moderately to 
abnormally high food prices due to the spillover effects of conflicts, such 
as disrupted supply chains. 
•	
	Achieving zero hunger requires intensified efforts to transform food 
systems so they are sustainable, resilient and equitable. Moreover, 
accelerating improvements in diets, nutrition, health and hygiene is 
crucial to meeting the target of halving the number of children suffering 
f

In [11]:
topic_g2 = "End hunger, achieve food security and improved nutrition and promote sustainable agriculture"

result = extract_arguments_json(text_g2, topic_g2)
print(result)

Model Output: {"in_favor": ["Accelerated government spending and aid for agriculture can help address economic challenges", "Increased investment in agricultural development projects can improve food security", "Agricultural policy initiatives can enhance administrative management and financial services"], "against": ["Global government spending on agriculture relative to the sector’s GDP contribution has declined", "The COVID-19 pandemic has diverted government resources away from agriculture", "Non-concessional loans for agricultural development projects may exacerbate debt burdens in developing countries"]}
in_favor=['Accelerated government spending and aid for agriculture can help address economic challenges', 'Increased investment in agricultural development projects can improve food security', 'Agricultural policy initiatives can enhance administrative management and financial services'] against=['Global government spending on agriculture relative to the sector’s GDP contribution

In [12]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal2_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 3: Ensure healthy lives and promote well-being for all at all ages**

In [13]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g3 = extract_text_from_pdf(pdf_path, [13, 14, 15, 16])
print(text_g3)

The Sustainable Development Goals Report 2024
12
Good health 
and well-being
•	
The COVID-19 pandemic has ravaged global health, undoing nearly 10 
years of progress in life expectancy. While most health-related indicators 
are moving in the right direction globally, current trends are insufficient 
to meet targets set for 2030.
•	
The maternal mortality ratio is largely stuck at a level more than three 
times the 2030 target. Under-5 deaths reached an historic low in 2022, 
but progress has slowed. Without accelerated declines, 35 million 
children will not live to see their fifth birthday by 2030. 
•	
The global response to infectious diseases has saved millions of lives 
and paved the way for healthier communities. Inequalities and emerging 
threats jeopardize these accomplishments, however, underscoring the 
need for sustained efforts and innovative strategies.
•	
	More than half the world’s population is not covered by essential health 
services, while an ageing health workforce m

In [14]:
topic_g3 = "Ensure healthy lives and promote well-being for all at all ages"

result = extract_arguments_json(text_g3, topic_g3)
print(result)

Model Output: {"in_favor": [
  "Total net official development assistance for medical research and basic health sectors has significantly increased in recent years.",
  "ODA for basic health care from all donors more than doubled in real terms between 2015 and 2022.",
  "Ninety-three per cent of the burden of air pollution fell on low- and middle-income countries in 2019, highlighting the need for increased assistance.",
  "Official development assistance for health has scaled up, with a major share spent on COVID-19."
], "against": [
  "Despite the increase in official development assistance, hundreds of millions of people are going without needed treatment and care for alcohol use disorders.",
  "Air pollution caused approximately 104 deaths per 100,000 people annually by 2019, indicating a need for more effective solutions.",
  "The high mortality rates attributable to air pollution in Oceania, sub-Saharan Africa, and most of Asia suggest that current efforts may be insufficient.",


In [15]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal3_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 4: Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all**

In [16]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g4 = extract_text_from_pdf(pdf_path, [17, 18])
print(text_g4)

The Sustainable Development Goals Report 2024
16
Quality education
•	
	Progress towards Goal 4 has been slow since 2015, with only 58 per 
cent of students worldwide achieving a minimum proficiency in reading 
by 2019. Recent assessments reveal a significant decline in math and 
reading scores in many countries, highlighting a set of factors beyond the 
COVID-19 pandemic’s impact on global education. 
•	
Many countries face challenges such as inadequate education 
infrastructure, teacher shortages and insufficient teacher training. While 
technology has expanded educational opportunities, it has also widened 
inequalities, leaving millions of people, especially in marginalized and 
low-income communities, without access to education.
•	
To meet national 2030 education targets, which have been scaled back 
compared to the original Goal 4 targets, countries must annually enrol 
1.4 million children in early childhood education, admit a new child 
to school every two seconds until 2030 an

In [17]:
topic_g4 = "Ensure inclusive and equitable quality education"

result = extract_arguments_json(text_g4, topic_g4)
print(result)

Model Output: {"in_favor": [
  "Globally, only half of primary schools and just 62 per cent of secondary schools have basic infrastructure for students with disabilities.",
  "Between 2021 and 2022, the growth rate in schools’ Internet access for pedagogical purposes doubled globally and even tripled in some regions.",
  "At the upper secondary level, 91 per cent of schools had access to electricity, 81 per cent had computers and 69 per cent were connected to the Internet.",
  "The International Standard Classification of Teacher Training Programmes was endorsed in 2021 to define a minimum global standard for trained teachers."
], "against": [
  "Lags in basic services in schools continue to marginalize girls and students with disabilities.",
  "Globally, over 20 per cent of primary schools have no separate sanitation facilities for girls.",
  "A lack of handwashing facilities in 2 out of 10 schools poses health risks, especially during pandemics.",
  "Sub-Saharan Africa faces the stee

In [18]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal4_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 5: Achieve gender equality and empower all women and girls**

In [19]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g5 = extract_text_from_pdf(pdf_path, [19, 20])
print(text_g5)

The Sustainable Development Goals Report 2024
18
•	
The world continues to lag in its pursuit of gender equality by 2030. 
Harmful practices are decreasing but not at a rate keeping up with 
population growth. One in five girls still marry before age 18. A 
staggering 230 million girls and women have been subjected to female 
genital mutilation. Far too many women still cannot realize the right to 
decide on their sexual and reproductive health. Violence against women 
persists, disproportionately affecting those with disabilities.
•	
Parity in women’s participation in public life remains elusive, and in 
management positions, at current rates, parity will require another 176 
years. Women carry an unfair burden of unpaid domestic and care work, 
spending 2.5 times more hours a day on it than men.
•	
Strong and sustained commitments to changing biased social norms, 
eliminating harmful practices and abolishing discriminatory laws are 
urgently needed. Enhancing women’s roles in leaders

In [20]:
topic_g5 = "Achieve gender equality and empower all women and girls"

result = extract_arguments_json(text_g5, topic_g5)
print(result)

Model Output: {"in_favor": [
  "Women's empowerment in sexual and reproductive health decisions remains restricted globally, although positive strides are evident in some places",
  "New data from 32 countries show that 19 have seen a positive trend in women's ability to make decisions on sexual and reproductive health, particularly in Eastern and Southern Africa",
  "Data from 69 countries show that 56 per cent of married or in-union women aged 15–49 can decide on their sexual and reproductive health and rights"
], "against": [
  "Although 89 per cent of women can autonomously decide to use contraception, one in four cannot make health-care decisions or say no to sex",
  "Western and Central Africa has experienced notable declines in women's ability to make decisions on sexual and reproductive health",
  "Socioeconomic factors such as household wealth, education and place of residence may shape these outcomes, contributing to poverty, inequality and precariousness"
]}
in_favor=["Women

In [21]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal5_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 6: Ensure availability and sustainable management of water and sanitation for all**

In [22]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g6 = extract_text_from_pdf(pdf_path, [21, 22])
print(text_g6)

The Sustainable Development Goals Report 2024
20
Clean water and 
sanitation
•	
Despite some improvements, progress in water and sanitation remains 
insufficient. At the current speed, in 2030, 2 billion people will still live 
without safely managed drinking water, 3 billion without safely managed 
sanitation and 1.4 billion without basic hygiene services. 
•	
In 2022, roughly half the world’s population experienced severe water 
scarcity for at least part of the year. One quarter faced “extremely 
high” levels of water stress. Climate change worsens these issues, 
posing significant risks to social stability. While 153 Member States 
share transboundary waters, only one region is on track to cover all 
transboundary rivers, lakes and aquifers with cooperation arrangements 
by 2030.
•	
	Sustainable management of water resources underpins prosperity and 
peace for all. It demands increased funding; more data-driven decision-
making; skilled workforce development; innovative technology,

In [23]:
topic_g6 = "Ensure availability and sustainable management of water and sanitation for all"

result = extract_arguments_json(text_g6, topic_g6)
print(result)

Model Output: {"in_favor": [
  "Integrated water management boosts resilience to multiple crises, including climate change",
  "Countries with higher levels of integrated water resources management are more resilient to climate change, biodiversity loss and pollution",
  "Mainstreaming water resources in climate planning and financing allows them to build resilience and advance multiple development objectives"
], "against": [
  "Achieving coverage of all transboundary basins by 2030 will require significantly accelerated progress",
  "Global progress in implementing integrated water resources management remains slow",
  "Water stress is increasing, with devastating consequences for sustainable development"
]}
in_favor=['Integrated water management boosts resilience to multiple crises, including climate change', 'Countries with higher levels of integrated water resources management are more resilient to climate change, biodiversity loss and pollution', 'Mainstreaming water resources in 

In [24]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal6_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 7: Ensure access to affordable, reliable, sustainable and modern energy for all**

In [25]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g7 = extract_text_from_pdf(pdf_path, [23, 24])
print(text_g7)

The Sustainable Development Goals Report 2024
22
Affordable and clean 
energy
•	
Considerable strides have been made in achieving sustainable energy 
targets. The number of people lacking access to electricity dropped from 
958 million in 2015 to 685 million in 2022. The number without clean 
cooking fuels declined from 2.8 billion to 2.1 billion over the same period. 
Global capacity to generate electricity from renewable energy has begun 
expanding at an unprecedented rate, a trend expected to continue.
•	
Recent events, such as the COVID-19 pandemic and the war in Ukraine, 
have disrupted progress, including by exacerbating already insufficient 
international financial flows for clean energy in developing countries. 
Without a change in course, an estimated 660 million people will still 
lack electricity access and around 1.8 billion will go without clean cooking 
fuels and technologies by 2030.
•	
To achieve universal access to clean energy by 2030, robust policies 
are needed to a

In [26]:
topic_g7 = "Ensure access to affordable, reliable, sustainable and modern energy for all"

result = extract_arguments_json(text_g7, topic_g7)
print(result)

Model Output: {"in_favor": [
  "Investing in renewable energy can create jobs and stimulate local economies.",
  "Renewable energy can reduce greenhouse gas emissions and help mitigate climate change.",
  "Energy efficiency improvements can save consumers money on their energy bills.",
  "International public financial flows for clean energy have rebounded in 2022.",
  "Global investment in energy efficiency would need to triple by 2030 to meet the Goal 7 energy efficiency target."
], "against": [
  "The current rate of expansion of renewable energy is not sufficient to meet global demands.",
  "Disparities in access to sustainable energy are evident, with some countries having much lower installed capacity per capita.",
  "High interest rates pose a challenge for financing new clean energy projects, particularly in emerging markets and developing economies.",
  "Global primary energy intensity improvement rate has remained under the average observed during the last five years.",
  "In

In [27]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal7_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 8: Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all**

In [28]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g8 = extract_text_from_pdf(pdf_path, [25, 26])
print(text_g8)

The Sustainable Development Goals Report 2024
24
Decent work and 
economic growth
•	
Progress towards Goal 8 faces headwinds from the COVID-19 aftermath, 
trade tensions, rising debt in developing countries, conflicts and 
geopolitical strains – collectively threatening global economic growth. 
While global real GDP per capita growth rebounded in 2021, it slowed in 
2022 and is projected to stabilize through 2025.
•	
	Global unemployment hit a historic low of 5 per cent in 2023 yet 
persistent roadblocks remain in achieving decent work. Women and 
youth face higher unemployment rates. Informal employment poses a 
significant global challenge, with over 2 billion workers in informal jobs 
lacking social protection in 2023. In the LDCs, and in sub-Saharan Africa 
and Central and Southern Asia, nearly 9 in 10 workers are informally 
employed. Alarmingly, over one in five young people are not in education, 
employment or training (NEET). Compliance with fundamental labour 
rights has deter

In [29]:
topic_g8 = "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all"

result = extract_arguments_json(text_g8, topic_g8)
print(result)

Model Output: {"in_favor": [
  "Tourism remains crucial for jobs, local ecosystems, peacebuilding and poverty reduction through income generation and tax revenues.",
  "The sector rebounded to 82 per cent of its 2019 level, contributing to 3.1 per cent of global GDP.",
  "Women experienced a disproportionate increase in informal employment during the pandemic, highlighting the need for support."
], "against": [
  "Challenges such as inflation, climate change, geopolitical conflicts and uneven economic recovery threaten the sustainable development of the sector.",
  "The COVID-19 pandemic inflicted unprecedented damage on tourism, with its contribution to the global economy plummeting from 3.8 per cent of GDP in 2019 to 1.8 per cent in 2020.",
  "Small island developing States rely heavily on tourism and face significant challenges in recovering from the pandemic."
]}
in_favor=['Tourism remains crucial for jobs, local ecosystems, peacebuilding and poverty reduction through income genera

In [30]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal8_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 9: Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation**

In [31]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g9 = extract_text_from_pdf(pdf_path, [27, 28])
print(text_g9)

The Sustainable Development Goals Report 2024
26
Industry, innovation and 
infrastructure
•	
	Since 2022, manufacturing sector growth has plateaued at around 
2.7 per cent, a level expected to persist into 2024 due to the lingering 
impacts of the COVID-19 pandemic and geopolitical tensions. The 
medium- and high-tech sector maintained its manufacturing value 
added share at around 46 per cent in 2021. The air transport industry is 
rebounding strongly. Small enterprises continue to face hurdles such as 
limited access to credit, especially in sub-Saharan Africa and the LDCs; 
only around 17 per cent has access to loans or credit lines.
•	
	Despite reduced carbon dioxide (CO2) emissions intensity, global 
emissions have hit a record high. The mobile broadband coverage gap 
persists at 5 per cent. 
•	
	To accelerate progress towards Goal 9, a concerted effort is needed to 
promote inclusive and sustainable industrialization, foster innovation 
through increased investment in research an

In [32]:
topic_g9 = "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation"

result = extract_arguments_json(text_g9, topic_g9)
print(result)

Model Output: {"in_favor": [
  "The proportion of medium- and high-tech manufacturing value added in total value added rose from 46.34 per cent in 2015 to 46.88 per cent in 2019.",
  "The crisis highlighted the resilience of higher-technology industries, which experienced only a marginal decline of 0.67 percentage points in 2021.",
  "Prioritizing investments in education and skills development is crucial to expedite the growth of medium- and high-tech industries by equipping the workforce to compete effectively in a dynamic global manufacturing landscape.",
  "Developing strategies for low-carbon energy and green manufacturing calls for collaboration and approaches tailored to each country’s unique strengths."
], "against": [
  "Regional disparities exist in shares of medium- and high-tech manufacturing, with sub-Saharan Africa representing just 18.3 per cent of total manufacturing in 2021.",
  "Progress in reducing emissions intensity to cut global emissions overall has been insuffic

In [33]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal9_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 10: Reduce inequality within and among countries**

In [34]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g10 = extract_text_from_pdf(pdf_path, [29, 30])
print(text_g10)

The Sustainable Development Goals Report 2024
28
Reduced inequalities
•	
Despite the economic disruptions of the pandemic, the global share of 
people living on less than half the median income has been declining 
due to social assistance programmes. However, workers’ wages have not 
kept pace with productivity, and labour’s share of GDP has resumed its 
long-term decline.
•	
A historic reversal is threatening improvements in inequality among 
countries. The economies of half the world’s most vulnerable countries 
have been growing at slower rates than those of wealthy countries. 
•	
	More people died on migration routes in 2023 than in any other year on 
record. The number of refugees worldwide reached a historic high. 
•	
	Developing countries are not fairly represented in international economic 
decision-making. Strengthening their voice and participation is crucial to 
ensuring a more inclusive and equitable global economic system.
•	
Addressing inequality both within and among cou

In [35]:
topic_g10 = "Reduce inequality within and among countries"

result = extract_arguments_json(text_g10, topic_g10)
print(result)

Model Output: {"in_favor": [
  "The number of refugees has increased year-on-year for more than a decade and doubled in less than seven years.",
  "Digital remittance services could help reach the target faster.",
  "Regulators need to address challenges such as limited competition, high compliance costs, account access barriers, transparency issues, and low digital and financial literacy."
], "against": [
  "2023 saw the highest number of refugees and was the deadliest year on record for migrants.",
  "The cost of remittances has fallen significantly but is still above the 3 per cent SDG target.",
  "Data collection challenges and limited responses mean that the actual number of migrant deaths is likely much higher."
]}
in_favor=['The number of refugees has increased year-on-year for more than a decade and doubled in less than seven years.', 'Digital remittance services could help reach the target faster.', 'Regulators need to address challenges such as limited competition, high compl

In [36]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal10_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 11: Make cities and human settlements inclusive, safe, resilient and sustainable**

In [37]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g11 = extract_text_from_pdf(pdf_path, [31, 32])
print(text_g11)

The Sustainable Development Goals Report 2024
30
Sustainable cities and 
communities
•	
Over half the world’s population currently resides in cities, yet urban 
areas confront myriad complex challenges. Globally, approximately 
one quarter of the urban population lives in slums, with the total slum 
population reaching 1.1 billion in 2022. The lack of equitable access to 
public transportation is a significant concern, particularly in LDCs, where 
only 4 in 10 individuals have convenient access. 
•	
	While air pollution levels have declined in most regions, they are still 
significantly higher than the recommended air quality guidelines for 
public health protection.
•	
Only 40 per cent of city dwellers can easily reach open public spaces. 
Between 2000 and 2020, cities sprawled up to 3.7 times faster than 
they densified, resulting in negative impacts on the natural environment 
and land use.
•	
With urbanization on the rise and nearly 70 per cent of the global 
population projected t

In [38]:
topic_g11 = "Make cities and human settlements inclusive, safe, resilient and sustainable"

result = extract_arguments_json(text_g11, topic_g11)
print(result)

Model Output: {"in_favor": [
  "Accelerated investments are needed to extend public transit services to the most marginalized communities.",
  "Open public spaces in urban areas foster inclusive and vibrant cities.",
  "Concerted efforts are necessary to ensure the majority of urban populations have access to vital amenities.",
  "Investments in urban development can improve the livelihoods of the urban poor."
], "against": [
  "Many cities worldwide lack adequate urban transit infrastructure.",
  "The shortfall in public transportation options is particularly problematic in the LDCs.",
  "Too few residents have ready access to open public spaces.",
  "Unaddressed, the majority of urban populations will struggle to access vital amenities."
]}
in_favor=['Accelerated investments are needed to extend public transit services to the most marginalized communities.', 'Open public spaces in urban areas foster inclusive and vibrant cities.', 'Concerted efforts are necessary to ensure the majori

In [39]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal11_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 12: Ensure sustainable consumption and production patterns**

In [40]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g12 = extract_text_from_pdf(pdf_path, [33, 34])
print(text_g12)

The Sustainable Development Goals Report 2024
32
Responsible 
consumption and 
production
•	
Countries have made strides in meeting obligations under international 
environmental agreements on hazardous waste and other chemicals 
and implementing comprehensive approaches to combat environmental 
degradation. Patterns of unsustainable consumption and production 
persist, however. In 2022, global food waste reached 1.05 billion metric 
tons, yet only 9 of 193 countries included food waste in their nationally 
determined contributions (NDCs) on climate change actions. The rapid 
growth of global e-waste remains largely unaddressed, with only 22 per 
cent collected and managed sustainably.
•	
While domestic material consumption and material footprints continue 
to expand, growth rates have slowed. Regional disparities underscore the 
need for targeted interventions based on varying consumption patterns 
and environmental impacts.
•	
Achieving Goal 12 requires fostering circular economy mod

In [41]:
topic_g12 = "Ensure sustainable consumption and production patterns"

result = extract_arguments_json(text_g12, topic_g12)
print(result)

Model Output: {"in_favor": [
  "Reducing food waste can help reduce greenhouse gas emissions and mitigate climate change.",
  "Increasing efficiency in material consumption can lead to cost savings for businesses and individuals.",
  "Implementing sustainable tourism practices can generate jobs and stimulate local economies.",
  "Improving monitoring and reporting of environmental impacts can inform policy decisions and drive positive change."
], "against": [
  "The growth rate of domestic material consumption has slowed, but absolute consumption levels remain high.",
  "Regional disparities in material consumption persist, with some regions experiencing faster growth than others.",
  "The use of Tourism Satellite Accounts and System of Environmental Economic Accounts has declined in recent years.",
  "Despite progress, more work is needed to achieve sustainable development goals and reduce environmental pressures."
]}
in_favor=['Reducing food waste can help reduce greenhouse gas emiss

In [42]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal12_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 13: Take urgent action to combat climate change and its impacts**

In [43]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g13 = extract_text_from_pdf(pdf_path, [35, 36])
print(text_g13)

The Sustainable Development Goals Report 2024
34
Reaching the $100 billion climate finance goal is a milestone yet trillions are needed for national action 
plans
Climate finance is crucial in backing global mitigation and adaptation 
efforts. Developed countries committed to mobilize $100 billion annually 
in climate finance for developing countries by 2020 and through 2025. The 
Organisation for Economic Co-operation and Development (OECD) reports 
that the commitment was met for the first time in 2022. Climate finance 
increased by 30 per cent from 2021 to reach $115.9 billion in 2022, with 60 
per cent of the total allocated to mitigation. Total adaptation finance rose to 
$32.4 billion from $10.1 billion in 2016. The 2021 Glasgow Climate Pact urged 
developed nations to double adaptation finance to developing countries from 
2019 levels by 2025. Based on OECD figures, by 2022, developed countries 
were approximately halfway towards meeting this goal.
Negotiations are under way to 

In [44]:
topic_g13 = "Take urgent action to combat climate change and its impacts"

result = extract_arguments_json(text_g13, topic_g13)
print(result)

Model Output: {"in_favor": [
  "Improved preparedness, including early warning systems and evacuation measures, have significantly reduced disaster mortality in recent years.",
  "National long-term risk reduction strategies and plans, reported by 129 countries; the establishment of effective multi-hazard early warning systems, reported by 104 countries; and comprehensive resilience plans are key to minimizing disaster impacts on people and economies."
], "against": [
  "Fossil fuel subsidies reached a record high of $1.53 trillion in 2022, driven by rising energy prices post-pandemic and the impact of the Russian Federation’s invasion of Ukraine, which disrupted global energy markets.",
  "The increase widened the gap between consumer prices and actual energy costs, prompting many governments to introduce support measures to shield industries and populations from inflation.",
  "By artificially lowering fossil fuel prices, subsidies harm the competitiveness of cleaner energy sources a

In [45]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal13_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 14: Conserve and sustainably use the oceans, seas and marine resources for sustainable development**

In [46]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g14 = extract_text_from_pdf(pdf_path, [37, 38])
print(text_g14)

The Sustainable Development Goals Report 2024
36
Life below water
•	
Oceans face significant challenges from eutrophication, worsening 
acidification, declining fish stocks, rising temperatures and widespread 
pollution. All these factors destroy habitats, diminish biodiversity and 
threaten coastal communities and the health of marine ecosystems, vital 
to over 3 billion people.
•	
Efforts to address these mounting concerns remain uneven. Key actions 
include implementing sustainable fishing practices, expanding marine 
protected areas to safeguard key biodiversity areas, increasing capacities 
to monitor ocean health and addressing the pollution that is choking 
waterways.
•	
Comprehensive global action is under way, yet it must accelerate. 
Priorities include ensuring that the Agreement on Fisheries Subsidies 
enters into force as soon as possible; increasing participation in the 
Agreement on Port State Measures to combat illegal, unreported and 
unregulated fishing; adopting a glo

In [47]:
topic_g14 = "Conserve and sustainably use the oceans, seas and marine resources for sustainable development"

result = extract_arguments_json(text_g14, topic_g14)
print(result)

Model Output: {"in_favor": [
  "The Agreement on Marine Biodiversity of Areas Beyond National Jurisdiction aims to protect marine biodiversity in international waters and seabed.",
  "Adequate management of protected areas will be crucial to realize conservation benefits.",
  "Marine designations should be strategically located to safeguard key biodiversity areas.",
  "The survival of coral reefs is vital for ocean health and humanity’s well-being."
], "against": [
  "Rising ocean temperatures are raising new concerns for coral reefs, which support a quarter of marine species.",
  "Up to 90 per cent of corals could be lost by 2050 at 1.5°C of warming and up to 99 per cent at 2°C.",
  "Less than 30 per cent of marine key biodiversity areas in Oceania (excluding Australia and New Zealand), Central and Southern Asia, and Northern Africa and Western Asia are protected."
]}
in_favor=['The Agreement on Marine Biodiversity of Areas Beyond National Jurisdiction aims to protect marine biodivers

In [48]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal14_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 15: Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, and halt and reverse land degradation and halt biodiversity loss**

In [49]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g15 = extract_text_from_pdf(pdf_path, [39, 40])
print(text_g15)

The Sustainable Development Goals Report 2024
38
•	
Global trends underscore persistent challenges to biodiversity and 
forests, despite their critical roles as planetary life-support systems. 
Global forest area continues to decline, primarily due to agricultural 
expansion, despite notable progress in sustainable forest management. 
Alarmingly, species are silently becoming extinct, the protection of key 
biodiversity areas has stalled and global illicit wildlife trafficking has 
steadily increased, posing serious threats to biodiversity and the benefits 
it provides to people.
•	
Efforts are under way to tackle these challenges, with countries 
advancing implementation of access and benefit-sharing instruments 
and integrating biodiversity values into national accounting systems. 
There’s also a growing global commitment to biodiversity conservation, 
reflected in increased funding and the adoption of the Kunming-
Montreal Global Biodiversity Framework.
•	
Urgent action is imperativ

In [50]:
topic_g15 = "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, and halt and reverse land degradation and halt biodiversity loss"

result = extract_arguments_json(text_g15, topic_g15)
print(result)

Model Output: {"in_favor": [
  "Detrimental land cover changes are impacting mountain ecosystems",
  "Mountain ecosystem degradation threatens downstream water provision",
  "Diminished forest and vegetation coverage increase soil erosion",
  "Globally, only about a third of mountain key biodiversity areas are safeguarded by protected areas or other effective area-based conservation measures"
], "against": [
  "The widespread transformation of mountain ecosystems is due to various natural and human-induced factors",
  "Changes in land use and coverage significantly contribute to biodiversity loss",
  "At least 100 million hectares of productive land were degraded annually between 2015 and 2019"
]}
in_favor=['Detrimental land cover changes are impacting mountain ecosystems', 'Mountain ecosystem degradation threatens downstream water provision', 'Diminished forest and vegetation coverage increase soil erosion', 'Globally, only about a third of mountain key biodiversity areas are safeguar

In [51]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal15_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 16: Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels**

In [52]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g16 = extract_text_from_pdf(pdf_path, [41, 42])
print(text_g16)

The Sustainable Development Goals Report 2024
40
Peace, justice and 
strong institutions
•	
Rising conflicts and violent organized crime persist around the world, 
causing immense human suffering and hampering sustainable 
development. The number of forcibly displaced people reached an 
unprecedented 120 million in May 2024. Civilian casualties in armed 
conflicts surged by 72 per cent in 2023.
•	
Corruption continues to divert resources from sustainable development, 
with one in five people reporting being asked to pay or having paid a 
bribe to a public official in the last 12 months. With one third of prisoners 
unsentenced worldwide, achieving just judicial processes and improving 
prison conditions is imperative. 
•	
Governments globally are struggling to restore pre-COVID-19 budget 
reliability. New international pressures affecting economies have 
increased budget expenditures beyond initial approvals. 
•	
All SDGs depend on establishing lasting peace and preventing violent 
con

In [53]:
topic_g16 = "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels"

result = extract_arguments_json(text_g16, topic_g16)
print(result)

Model Output: {"in_favor": [], "against": []}
in_favor=[] against=[]


In [54]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal16_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

### **Goal 17: Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development**

In [55]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text_g17 = extract_text_from_pdf(pdf_path, [43, 44])
print(text_g17)

The Sustainable Development Goals Report 2024
42
Partnerships for the 
Goals
•	
Developing countries face a $4 trillion annual investment gap to achieve 
the SDGs. Foreign direct investment flows to developing countries have 
decreased while gains in remittances and ODA have been modest. The 
Secretary-General has urged countries to support an SDG Stimulus of at 
least $500 billion annually.
•	
External debt stock levels have remained unprecedentedly high in 
developing countries. About 60 per cent of low-income countries are at 
high risk of debt distress or already experiencing it. 
•	
Approximately 67 per cent of the world’s population was online in 2023, 
up 69 per cent from 2015. Investment in infrastructure and affordable 
Internet access for the remaining 2.6 billion people is crucial for inclusive 
development.
•	
The COVID-19 pandemic highlighted the importance of high-quality, 
timely and disaggregated data for informed decision-making. Yet 
funding gaps threaten the resilien

In [56]:
topic_g17 = "Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development"

result = extract_arguments_json(text_g17, topic_g17)
print(result)

Model Output: {"in_favor": [
  "Remittances can help alleviate poverty in developing countries by providing a source of income for families and individuals.",
  "International cooperation and investment in statistical capacity can help bridge the data gap and support sustainable development.",
  "Investments in infrastructure and affordable Internet access are crucial for increasing Internet adoption rates, especially in underserved regions.",
  "The COVID-19 pandemic has highlighted the importance of reliable and timely data for informed decision-making.",
  "Increased funding for data and statistics development can help address funding gaps and support the resilience of statistical systems."
], "against": [
  "The digital divide persists, with one in three people still offline, underscoring the urgency of infrastructure investments and affordable Internet access.",
  "Funding gaps remain a major issue, particularly in Africa and the LDCs, where only a fraction of plans is fully funde

In [57]:
data_dict = result.model_dump()

with open("./results_gpt/arguments_goal17_base.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)